In [1]:
import covidcast as covid
import pandas as pd

### Fetch raw covid data and save to csv

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# confirmed_cumulative_cases = covid.signal(data_source="usa-facts",
#                                           signal="confirmed_cumulative_num",
#                                           geo_type="county")

In [ ]:
# confirmed_daily_incidence_cases = covid.signal(data_source="usa-facts",
#                                                signal="confirmed_incidence_num",
#                                                geo_type="county")

In [ ]:
# cumulative_deaths = covid.signal(data_source="usa-facts",
#                                  signal="deaths_cumulative_num",
#                                  geo_type="county")

In [ ]:
# daily_incidence_deaths = covid.signal(data_source="usa-facts",
#                                       signal="deaths_incidence_num",
#                                       geo_type="county")

In [ ]:
# confirmed_cumulative_cases.to_csv("data/covid_usafacts/raw/confirmed_cumulative_cases.csv")
# confirmed_daily_incidence_cases.to_csv("data/covid_usafacts/raw/confirmed_daily_incidence_cases.csv")
# cumulative_deaths.to_csv("data/covid_usafacts/raw/cumulative_deaths.csv")
# daily_incidence_deaths.to_csv("data/covid_usafacts/raw/daily_incidence_deaths.csv")

### Read raw covid data from local saved csv file

In [ ]:
# confirmed_cumulative_cases = pd.read_csv("data/covid_usafacts/raw/confirmed_cumulative_cases.csv", index_col=0)
# confirmed_daily_incidence_cases = pd.read_csv("data/covid_usafacts/raw/confirmed_daily_incidence_cases.csv", index_col=0)
# cumulative_deaths = pd.read_csv("data/covid_usafacts/raw/cumulative_deaths.csv", index_col=0)
# daily_incidence_deaths = pd.read_csv("data/covid_usafacts/raw/daily_incidence_deaths.csv", index_col=0)

### Retreive FIPS codes for each US county

In [ ]:
# fips = pd.read_excel('data/covid_usafacts/raw/area_fips_2019.xlsx', skiprows=range(4))
# state_fips = pd.read_excel("data/covid_usafacts/raw/state_fips_2019.xlsx", skiprows=range(5))

In [ ]:
# county_fips = fips[fips["Summary Level"]==50]

In [ ]:
# # Rename columns for convenience
# county_fips = county_fips.rename(columns={"Area Name (including legal/statistical area description)": "Area Name",
#                                           "State Code (FIPS)": "State FIPS",
#                                           "County Code (FIPS)": "County FIPS"})

In [ ]:
# # Convert Area Name column to title case
# county_fips["Area Name"] = county_fips["Area Name"].str.title()

In [ ]:
# # Get full FIPS code for each row
# county_fips["FIPS"] = county_fips["State FIPS"].apply(str).str.zfill(2) + county_fips["County FIPS"].apply(str).str.zfill(3)
# county_fips["FIPS"] = county_fips["FIPS"].astype('int64')

In [ ]:
# # Get state names for each row
# county_fips = county_fips[["State FIPS", "County FIPS", "FIPS", "Area Name"]]
# state_fips = state_fips[["State (FIPS)", "Name"]]

In [ ]:
# county_fips = county_fips.merge(state_fips, how='left', left_on="State FIPS", right_on="State (FIPS)")
# county_fips = county_fips.rename(columns={"Name": "State Name"})

In [ ]:
# county_fips = county_fips[["State FIPS", "County FIPS", "FIPS", "State Name", "Area Name"]]

In [ ]:
# county_fips.to_csv("data/covid_usafacts/clean/county_fips_2019.csv")

### Merge covid datasets with fips data

In [ ]:
# def merge_covid_fips_dfs(covid_df, fips_df):
#     df = covid_df.merge(fips_df, how='left', left_on='geo_value', right_on='FIPS')
#     return df[["geo_value", "time_value", "issue", "value", "State Name", "Area Name"]]

In [ ]:
# confirmed_cumulative_cases_fips = merge_covid_fips_dfs(confirmed_cumulative_cases, county_fips)
# confirmed_daily_incidence_cases_fips = merge_covid_fips_dfs(confirmed_daily_incidence_cases, county_fips)
# cumulative_deaths_fips = merge_covid_fips_dfs(cumulative_deaths, county_fips)
# daily_incidence_deaths_fips = merge_covid_fips_dfs(daily_incidence_deaths, county_fips)

In [ ]:
# confirmed_cumulative_cases_fips.to_csv("data/covid_usafacts/clean/confirmed_cumulative_cases_fips.csv")
# confirmed_daily_incidence_cases_fips.to_csv("data/covid_usafacts/clean/confirmed_daily_incidence_cases_fips.csv")
# cumulative_deaths_fips.to_csv("data/covid_usafacts/clean/cumulative_deaths_fips.csv")
# daily_incidence_deaths_fips.to_csv("data/covid_usafacts/clean/daily_incidence_deaths_fips.csv")

### Read clean covid and fips data from local saved csv files

In [2]:
confirmed_cumulative_cases_county = pd.read_csv("data/covid_usafacts/clean/confirmed_cumulative_cases_fips.csv", index_col=0)
confirmed_daily_incidence_cases_county = pd.read_csv("data/covid_usafacts/clean/confirmed_daily_incidence_cases_fips.csv", index_col=0)
cumulative_deaths_county = pd.read_csv("data/covid_usafacts/clean/cumulative_deaths_fips.csv", index_col=0)
daily_incidence_deaths_county = pd.read_csv("data/covid_usafacts/clean/daily_incidence_deaths_fips.csv", index_col=0)

In [ ]:
# all_data = covid.signal(data_source="usa-facts",
#                     signal="confirmed_cumulative_num", # signal="confirmed_7dav_incidence_prop",
#                     start_day=date(2020, 8, 1),  # date.today() - timedelta(days=2)
#                     end_day=date(2020, 8, 2),  # end_day=date.today()
#                     geo_type="county")

### Fetch and clean USDA County-level data

#### New York 

In [173]:
ny_poverty = pd.read_excel("data/usda_county_datasets/poverty_percentages_2018/ny_counties.xlsx")
ny_poverty = ny_poverty[["FIPS", "Name", "Total Percent", "Children Percent"]].rename(
    columns={"Total Percent": "% Population in Poverty (2018)", "Children Percent": "% Children in Poverty (2018)" })

In [174]:
ny_unemployment_median_hhi = pd.read_excel("data/usda_county_datasets/unemployment_rate_median_hhi_2018/ny_counties.xlsx", skiprows=range(1))
ny_unemployment_median_hhi = ny_unemployment_median_hhi.dropna(how='all')
ny_unemployment_median_hhi["FIPS"] = ny_unemployment_median_hhi["FIPS"].astype("int64")
ny_unemployment_median_hhi = ny_unemployment_median_hhi[["FIPS", 2018, "Median Household Income (2018)", "% of State Median HH Income"]]
ny_unemployment_median_hhi = ny_unemployment_median_hhi.rename(columns={2018: "Unemployment Rate (2018)"})

In [175]:
def get_recent_education_data(csv_path, education_col_name):
    df = pd.read_excel(csv_path, skiprows=range(2))
    df = df[["FIPS", "2014-2018"]]
    df["2014-2018"] = df["2014-2018"] * 100
    df = df.rename(columns = {"2014-2018": "% " + education_col_name + " (2014-2018)"})
    return df

In [176]:
ny_complete_college = get_recent_education_data("data/usda_county_datasets/education_1970_2018/complete_college/ny_counties.xlsx", 
                          "Complete College")

ny_complete_high_school = get_recent_education_data("data/usda_county_datasets/education_1970_2018/complete_high_school/ny_counties.xlsx", 
                          "Complete High School")

ny_incomplete_high_school = get_recent_education_data("data/usda_county_datasets/education_1970_2018/incomplete_high_school/ny_counties.xlsx", 
                          "Incomplete High School")

ny_education = ny_complete_college.merge(ny_complete_high_school, how='outer', on="FIPS").merge(ny_incomplete_high_school, how='outer', on="FIPS")

In [177]:
ny_usda_data = ny_poverty.merge(ny_unemployment_median_hhi, how='outer', on='FIPS').merge(ny_education, how='outer', on='FIPS')

In [178]:
ny_usda_data.to_csv("data/usda_county_datasets/clean/ny_counties.csv")

#### Pennsylvania

In [179]:
pa_poverty = pd.read_excel("data/usda_county_datasets/poverty_percentages_2018/pa_counties.xlsx")
pa_poverty = pa_poverty[["FIPS", "Name", "Total Percent", "Children Percent"]].rename(columns={
    "Total Percent": "% Population in Poverty (2018)", 
    "Children Percent": "% Children in Poverty (2018)" 
})

In [180]:
pa_unemployment_median_hhi = pd.read_excel("data/usda_county_datasets/unemployment_rate_median_hhi_2018/pa_counties.xlsx", skiprows=range(1))
pa_unemployment_median_hhi = pa_unemployment_median_hhi.dropna(how='all')
pa_unemployment_median_hhi["FIPS"] = pa_unemployment_median_hhi["FIPS"].astype("int64")
pa_unemployment_median_hhi = pa_unemployment_median_hhi[["FIPS", 2018, "Median Household Income (2018)", "% of State Median HH Income"]]
pa_unemployment_median_hhi = pa_unemployment_median_hhi.rename(columns={2018: "Unemployment Rate (2018)"})

In [181]:
pa_complete_college = get_recent_education_data("data/usda_county_datasets/education_1970_2018/complete_college/pa_counties.xlsx", 
                          "Complete College")

pa_complete_high_school = get_recent_education_data("data/usda_county_datasets/education_1970_2018/complete_high_school/pa_counties.xlsx", 
                          "Complete High School")

pa_incomplete_high_school = get_recent_education_data("data/usda_county_datasets/education_1970_2018/incomplete_high_school/pa_counties.xlsx", 
                          "Incomplete High School")

pa_education = pa_complete_college.merge(pa_complete_high_school, how='outer', on="FIPS").merge(pa_incomplete_high_school, how='outer', on="FIPS")

In [182]:
pa_usda_data = pa_poverty.merge(pa_unemployment_median_hhi, how='outer', on='FIPS').merge(pa_education, how='outer', on='FIPS')

In [183]:
pa_usda_data.to_csv("data/usda_county_datasets/clean/pa_counties.csv")